<a href="https://colab.research.google.com/github/torzdf/faceswap/blob/colab/faceswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left">
  <a href="https://faceswap.dev"><img src="https://i.imgur.com/zHvjHnb.png"></img></a>Welcome to the FaceSwap Notebook
</p>



The FaceSwap Notebook is a means of running FaceSwap on Google Colab.

Colab will give you a free GPU for up to a 12 hour period (as long as your browser window remains open).

Please make sure you understand Google Colaboratory's terms of service. This notebook is for testing out Faceswap and experimenting. It is not for creating swaps on an industrial scale. [See here for more info](https://research.google.com/colaboratory/faq.html#gpu-availability).

At a minimum you must run the [Prerequisites](#scrollTo=GVzyxw4xkKI7). Once complete, you can jump to the section that your require.

**NB:** This notebook is provided as a courtesy. It is not a priority for the FaceSwap project so there is no guarantee that it will remain up to date and support is likely to be limited. Please be aware that this Notebook lacks some features from the full application.

>[Prerequisites](#scrollTo=GVzyxw4xkKI7)

>>[GPU Check](#scrollTo=_jYRKp5zkYMU)

>>[Link Google Drive](#scrollTo=Sb1aWvH3phjA)

>>[Get FaceSwap](#scrollTo=oql-3Tf7lquV)

>[Run Faceswap](#scrollTo=Fv8MBijaU3PK)



# Prerequisites
First up we need to make sure we have a GPU instance and set up our environment.

Whatever task you plan to perform, you must run these steps first.

## GPU Check
Run the following code block to make sure you have been allocated a GPU:

In [0]:
#@title
!nvidia-smi

 You should receive output similar to below:
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

If you have not been allocated a GPU then select `Runtime` > `Change runtime type` from the top menu and ensure that the `Runtime type` is `Python 3` and `Hardware Acceleration` is `GPU`

## Link Google Drive
Now we know we have a GPU instance, we need to link your Google Drive.

You will need to give Colab permissions to mount your drive, so run the code block below and follow the instructions to mount your drive:

In [0]:
#@title
import os
_ROOT="/content/drive/My Drive/_faceswap_colab"
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

if not os.path.exists(_ROOT):
  !mkdir "$_ROOT"

!echo -en "\e[32m"
print("Google Drive Linked")
!echo -en "\e[0m"

## Get FaceSwap
Time to tell Colab where to find/install the FaceSwap App. Select your options in the form, then hit play:

In [0]:
#@title App install location
#@markdown Where do you want Faceswap installed?
#@markdown * GoogleDrive: [Recommended] - Store the FaceSwap App on your Google
#@markdown Drive. This means any config files that are changed won't get lost
#@markdown when the instance destroys. However it will take up about 200MB of
#@markdown space.
#@markdown * Local: FaceSwap App will be stored in the Colab Notebook. The app
#@markdown will disappear when the instance dies.
location = "GoogleDrive"  #@param ['GoogleDrive', 'Local']
#@markdown Do You want to update an existing app or create a new one?
#@markdown * Update: [Recommended] - If the FaceSwap App is already installed in
#@markdown the chosen location, it will be updated. If the app doesn't already
#@markdown exist, it will be installed.
#@markdown * New: Uninstall any installed FaceSwap App and Reinstall for the
#@markdown chosen location.
app_update = "Update"  #@param ['New', 'Update']

import os
import shutil
import sys

_ROOT="/content/drive/My Drive/_faceswap_colab" if location == "GoogleDrive" else "."
_APPDIR=os.path.join(_ROOT, "faceswap")
_APP=os.path.join(_APPDIR, "faceswap.py")
fs_backend_path=os.path.join(_APPDIR, "config", ".faceswap")

print("\033[32mInstalling Tensorfow...\033[0m")
!pip install tensorflow-gpu==1.15

print("\033[32mApp Location:", _APP)

# Dummy in a backend config in root dir to prevent backend selector popping
sys_location = os.path.dirname(os.path.realpath(sys.argv[0]))
sys_conf = os.path.join(sys_location, "config", ".faceswap")
if not os.path.isdir(os.path.dirname(sys_conf)):
  os.mkdir(os.path.dirname(sys_conf))
  with open(sys_conf, "w") as sys_config:
    sys_config.write("{\"backend\": \"nvidia\"}")

if app_update == "New" or not os.path.isfile(_APP):

  if os.path.isdir(_APPDIR):
    print("Uninstalling pre-existing app...")
    shutil.rmtree(_APPDIR)

  print("Getting Faceswap...")
  !echo -en "\e[0m"
  !git clone --depth 1 -b master https://github.com/deepfakes/faceswap.git "$_APPDIR"
  %cd "$_APPDIR"

  print("\033[32mInstalling Dependencies. This may take a minute or two...\033[0m")
  !pip install -r "$_APPDIR/requirements.txt"

  print("\033[32mSetting backend to nvidia:\033[0m", fs_backend_path)
  with open(fs_backend_path, "w") as backend_config:
    backend_config.write("{\"backend\": \"nvidia\"}")
else:
  %cd "$_APPDIR"
  
  print("\033[32mUpdating App...\033[0m")
  !git pull 2>&1

  print("\033[32mSetting backend to nvidia:\033[0m", fs_backend_path)
  with open(fs_backend_path, "w") as backend_config:
    backend_config.write("{\"backend\": \"nvidia\"}")

  print("\033[32mUpdating Dependencies. This may take a minute or two...\033[0m")
  !python "$_APPDIR/update_deps.py"

print("\033[32mFaceswap Ready!\033[0m")


# Run Faceswap
Now we're all setup, it's time to choose a task.

Remember, you can edit the plugin configuration files here before proceeding:
`/content/drive/My Drive/_faceswap_colab/faceswap/config`

Select your option in the form, then hit play:

In [0]:
#@title Faceswap Task
#@markdown Which task do you want to run?
task = "Extract"  #@param ['Extract', 'Train', 'Convert']
#@markdown Once you're happy with your selection, run this code block to get task options.

from textwrap import wrap
from ipywidgets import widgets
def get_cliopts(section):
  """ obtain the cli options for the given section """
  import lib.cli as cli
  import inspect
  mod_class=[name for name, obj in inspect.getmembers(cli)
             if inspect.isclass(obj) and name.lower().endswith("args")
             and name.lower().startswith(section)
             and name.lower() != "extractconvertargs"][0]

  name = mod_class.lower().replace("args", "")
  meth = getattr(cli, mod_class)(None, name)
  print("\033[32m{}\033[0m".format(name.title()))
  print(meth.info.split("\n")[0], "\n")

  args = meth.argument_list + meth.optional_arguments + meth.global_arguments
  final_args = []
  for arg in args:
    action = arg.get("action", None)
    _opt_name = arg["opts"][-1].replace("--", "")
    if _opt_name in ("logfile", "gui", "Singleprocess"):
      continue
    if action == getattr(cli, "Radio"):
      arg["control"] = widgets.RadioButtons(
          options=arg.get("choices", []),
        value=arg.get("default", None),
          disabled=False)
    elif action == getattr(cli, "Slider") and arg["type"] == int:
      arg["control"] = widgets.IntSlider(
        value=arg.get("default", None),
        min=arg["min_max"][0],
        max=arg["min_max"][1],
        step=arg["rounding"],
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        options=arg.get("choices", []))
    elif action == getattr(cli, "Slider") and arg["type"] == float:
      arg["control"] = widgets.FloatSlider(
        value=arg.get("default", None),
        min=arg["min_max"][0],
        max=arg["min_max"][1],
        step=arg["rounding"] / 100,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f')
    elif action == "store_true":
      arg["control"] = widgets.Checkbox(
        value=arg.get("default", None),
        disabled=False)
    elif arg.get("choices", None) is not None:
      arg["control"] = widgets.Dropdown(
        options=arg["choices"],
        value=arg.get("default", None),
        disabled=False)
    elif arg.get("type", str) == int:
      arg["control"] = widgets.IntText(
        value=arg.get("default", None),
        disabled=False)
    elif arg.get("type", str) == float:
      arg["control"] = widgets.FloatText(
        value=arg.get("default", None),
        disabled=False)
    else:
      arg["control"] = widgets.Text(
        value=arg.get("default", None),
        placeholder='Type something',
        disabled=False)      
    final_args.append(arg)
  return final_args

_cli_opts = get_cliopts(task.lower())
_groups = dict()

for opt in _cli_opts:
  _groups.setdefault(opt.get("group", None), []).append(opt)

for group, opts in _groups.items():
  print("\033[31m{}\n== {}\n{}\033[0m".format("=" * 40, group, "=" * 40))
  for opt in opts:
    _opt_name = opt["opts"][-1].replace("--", "")
    print("-- \033[34m{}\033[0m {}".format(_opt_name, "-" * 100)[:109])
    display(opt["control"])

    _hlp = opt.get("help", "")
    if _hlp.startswith("R|"):
      line = _hlp[2:].replace("L|", "L|  - ")
      for row in line.split("L|"):
        for outp in wrap(row, 100):
          print(outp)
    else: 
      for line in wrap(opt.get("help", ""), 100):
        print(line)
    print("{}\n".format("-" * 100))
